
# 부동산 실거래가 데이터 크롤링

### --------------- 
## 1부_크롤링
### ---------------

### 01_지역 코드 불러오기

In [ ]:
# 01-01_InteractiveShell의 옵션 설정

# 주피터 노트북에서 여러 결과를 한번에 보여주기

from IPython.core.interactiveshell import InteractiveShell  
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# 01-02_라이브러리 불러오기

import requests      # HTTP 통신에 필요한 라이브러리
import os            # 운영체제(Operating System)에서 사용하는 기능을 구현한 라이브러리
import pandas as pd  # 데이터프레임을 다루기 위한 판다스 라이브러리
import datetime      # 날짜와 시간 계산에 사용하는 라이브러리
import time          # 시간관련 계산에 사용하는 라이브러리

In [ ]:
# 01-03_전국 시군구 코드 불러오기

code_list = pd.read_csv("./data/code_sigun.csv")
code_list

### 02_크롤링 지역 & 기간 선택

In [ ]:
# 02-01_데이터 수집지역 코드정보 추출

code_list = code_list[(code_list['sido'] == '서울특별시')]
code_list.tail(3)

In [ ]:
# 02-02_code_list 속성 알아보기

# 데이터프레임 내  변수의 형식(타입)을 알려줌

code_list.dtypes

In [ ]:
# 02-03_code_list 속성변경

# 코드값은 unique 값이어야 하므로 숫자가 아니라 문자로 사용하는 것이 적합
# 따라서 03_code_list 내 code 컬럼 속성을 int64(숫자)에서 string(문자)로 변경

code_list['code'] = code_list['code'].apply(str)      # code 속성 타입 변경: int64 => string
code_list.dtypes

In [ ]:
# 02-04a_자료수집 기간 설정하기(연도)

year = [str("%02d" %(y)) for y in range(2021, 2022)]  # 2021~ 2021년
year

In [ ]:
# 02-04b_자료수집 기간 설정하기(월)

month = [str("%02d" %(m)) for m in range(1, 13)]     # 1월 ~ 12월
month

In [ ]:
# 02-04c_자료수집 기간 설정하기(연월 결합)

base_date_list = ["%s%s" %(y, m) for y in year for m in month ]    # 연도/월별 데이터 리스트로 저장
str(base_date_list)                                                  # 변수를 문자열(structure) 형태로 출력

In [ ]:
# 02-05_인증키 받기

# 공공데이터포털에서 인증키 받기: https://www.data.go.kr/data/15058747/openapi.do

key = "iXXXD"
key

In [ ]:
# 02-06_사용자 함수(get_data) 정의: xml 형식의 데이터를 파싱(parsing)하기 위한 함수 만들기

# 파싱: 어떤 일정한 문법을 토대로 나열된 data들을 그 문법에 맞게 분석하여 새롭게 구성하는 작업
# 참고: 아파트 매매 신고정보 조회 기술문서.hwp

def get_data(code_list, v2):
    url ="http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?"
    service_key = key
    payload = "LAWD_CD=" + v1 + "&" + \
              "DEAL_YMD=" + v2 + "&" + \
              "serviceKey=" + service_key + "&"
    res = requests.get(url + payload)  # 요청 주소 + 요청 세부내역 활용하여 API 방법으로 정보 요청하기
    return res

In [ ]:
# 02-07_xml을 list 객체로 변환하기 위한 사용자 함수(get_items) 정의

# 파이썬에서 xml 을 파싱할때는 xml.etree.ElementTree 를 이용
# 주요 프로퍼티: tag, attrib, text
# 주요 메소드: iter(), findall()
import xml.etree.ElementTree as ET

def get_items(response):
    root = ET.fromstring(response.content)            # xml을 문자형(string)으로 읽어들여서 root에 저장
    item_list = []                                    # item_list라는 리스트형 empty dataset 생성
    #------#
    for child in root.find('body').find('items'):    # body 아래 있는 모든 item 찾기
                                                      # root - header / body - item
        elements = child.findall('*')                 # 모든 item 데이터 추출
        #------#
        data = {}                        # data라는 딕셔너리형 empty dataset 생성
        for element in elements:        # elements라는 변수 내에 포함되어 있는 태그와 텍스트 정리
            tag = element.tag.strip()    # 태그 제거: '<b>hello</b> => hello
            text = element.text.strip()  # 공백 제거: 텍스트 앞 뒤 공백제거
            data[tag] = text             # 정리된 tag와 text를 item_list 내에 누적(append)하여 저장
        item_list.append(data)  
    #-----#    
    return item_list

### 03_실거래가 크롤링 하기

In [ ]:
# 03-01_반복문 경계조건 설정

v1 = code_list['code'][0]        # 시작 지역코드(0번)
end_2 = len(base_date_list)      # 종료 지역 순서(반복횟수)
v2 = base_date_list[0]           # 시작연월
end_1 = len(code_list['code'])   # 종료 연월(반복횟수)

v1 ; v2 ; end_1 ; end_2

In [ ]:
# 03-02_크롤링 준비
items_list = []  # empty list(빈그릇) 만들기

number_1 = 0    # 반복문 초기화

In [ ]:
# 03-03_크롤링(2중 반복문)

for v1 in code_list['code'] :      # code_list(25개 지역)의 시작지역 - 종료지역까지 반복
    #-------#
    number_1 += 1                  # 반복문이 한번 끝날 때마다 1씩 누적 증가
    pcnt_1 = (number_1/end_1)*100  # 코드 리스트의 지역 중 몇 번째 지역까지 진행되었는지 계산
    number_2 = 0                   # 반복문이 한번 끝날 때마다 0으로 초기화
    #-------#
    for v2 in base_date_list:        # 한 지역의 시작연월 - 종료연월까지 반복  
     number_2 += 1                   # 반복문이 한번 끝날 때마다 1씩 누적 증가
     pcnt_2 = (number_2/end_2)*100   # 한 지역의 시작연월 - 종료연월 사이에 몇 번째 까지 진행되었는지 계산
     res = get_data(v1, v2)          # 앞에서 작성한 get_data 함수를 이용하여 xml자료 파싱
                                     # 이때 변수로 v1(지역코드)와 V2(연월) 변수를 활용
     items_list += get_items(res)    # 앞에서 작성한 get_items() 함수를 이용하여 수집된 xml 자료를 리스트 형식으로 변환하고
                                     # items_list라는 변수 내에 누적(+=)하여 저장
     #-------#
     time.sleep(.2)                  # 0.2초 쉬고 진행상황을 알려주는 메시지 출력(print)
     print("지역: (", number_1,"/",end_1,")",code_list['addr_1'][number_1-1],":", v1," [","%.2f%%" % pcnt_1,"] / 기간: ", v2,"[","%.1f%%" % pcnt_2,"]", end="\r")

In [ ]:
# 03-03_list 형식을 pandas dataframe 형태로 변경하기

items = pd.DataFrame(items_list) 
items

In [ ]:
# 03-04_해제사유 발생 거래건수 삭제

items = items[items['해제여부'] != 'O']
items.shape

In [ ]:
# 03-05_불필요한 컬럼 지우기

items = items.drop(["해제사유발생일", "해제여부"], axis=1)
items.shape

In [ ]:
# 03-06_Pickle로 데이터 저장

# pickle: 파이썬에서 사용하는 자료형을 변환 없이 그대로 저장하고 이를 불러올 때 사용하는 모듈
import pickle
items.to_pickle('03_price_DB.pkl')
items.to_csv('03_price_DB.csv',  encoding='euc-kr')

In [ ]:
# 03-05_메모리 클리어 

# 메뉴의 Kernel - Restart 클릭 => Restart(붉은색 단추) 누르기

### ---------------------- 
## 2부_데이터 전처리
### ----------------------

### 04_데이터 전처리

In [ ]:
# 04-00_주피터 노트북에서 여러 결과를 한번에 보여주기

from IPython.core.interactiveshell import InteractiveShell  
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# 04-01_저장한 피클 파일 불러오기 

import pandas as pd 
import pickle
items = pd.read_pickle('03_price_DB.pkl')
items.head(3)

In [ ]:
# 04-02_변수별 속성정보 확인

items.dtypes

In [ ]:
# 04-03_전국 시군구 코드 불러오기

code_list = pd.read_csv("./data/code_sigun.csv")
code_list['code'] = code_list['code'].apply(str)      # code 속성 타입 변경: int64 => string
code_list.columns = ['지역코드','시도','시군구','addr_1','addr_2',]
code_list.dtypes
code_list

In [ ]:
# 04-04_items + code_list 결합

items = pd.merge(items, code_list, how='left', left_on=['지역코드'], right_on=['지역코드'])
items.dtypes
items.head(4)

In [ ]:
# 4-05_주소 추가하기 

cols = ['시도','시군구','법정동', '지번', '아파트']
items['juso'] =items[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
items

In [ ]:
# 04-05_데이터프레임 속성보기

print(items.dtypes)

In [ ]:
# 04-05_콤마 지우기

def remove_comma(x):                 # 함수 정의
    return x.replace(',', '')

items['거래금액'] = items['거래금액'].apply(remove_comma)
items.head(2)

In [ ]:
# 04-06_속성변환: object(문자) -> float(숫자)

items["거래금액"] = items.거래금액.astype(float)    # pandas.Series.astype 방식 사용
items["전용면적"] = items.전용면적.astype(float)    # https://stackoverflow.com/questions/48094854/python-convert-object-to-float
items.dtypes

In [ ]:
# 04-07_평당가격 계산하기

items['평당'] = items["거래금액"] / items["전용면적"]  # 거래금액 / 면적 = 제곱미터당 가격
items['평당'] = round(items['평당'] * 3.3, 1)          # 제곱미터당 가격 X 3.3 
                                                       # round()함수로 소숫점 첫째 자리 표현
items.head(4)

In [ ]:
# 04-09_연, 월, 일 컬럼을 object -> int64로 변경

items['년'] = pd.to_numeric(items['년'])
items['월'] = pd.to_numeric(items['월'])
items['일'] = pd.to_numeric(items['일'])
items.dtypes

In [ ]:
# 04-10_YYYY MM DD를 => 시간형 YYYY-MM-DD로 바꾸기

items['날짜'] = pd.to_datetime(items.년*10000+items.월*100+items.일, format='%Y%m%d')
items.head(2)

In [ ]:
items.columns

In [ ]:
# 04-12_순서정리

items = items[list(('날짜', '년','월', '일','시도', '시군구', '법정동', '아파트', '층', '거래금액', '평당', '전용면적', '건축년도' , 'addr_2', 'juso'))]
items.head(2)

In [ ]:
# 04-13_저장하기 

items.to_pickle('04_apt_db.pkl')
items.to_csv('04_price_DB.csv',  encoding='euc-kr')

In [ ]:
# 04-14_메모리 클리어 

# 메뉴의 Kernel - Restart 클릭 => Restart(붉은색 단추) 누르기

### 05_공간 좌표값 부여 

In [ ]:
# 05-00_기본세팅

from IPython.core.interactiveshell import InteractiveShell  # 여러 결과를 한꺼번에 보여주기
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# 05-01_크롤링한 실거래가 데이터 불러오기

import pandas as pd 
import pickle
items = pd.read_pickle('04_apt_db.pkl')
items.head(2)

In [ ]:
# 04-02_실거래가 데이터 가운데 unique 주소만 남기기

apt_juso = items.drop_duplicates(['juso'], keep='first')
apt_juso = apt_juso['juso']
apt_juso

In [ ]:
# 04-03_인덱스 리셋하기

apt_juso = apt_juso.reset_index()
apt_juso = apt_juso.drop('index', axis=1)
apt_juso

In [ ]:
# 05-04_apt_juso => 좌표변환 확인하기

import requests
from urllib.parse import urlparse

address = apt_juso['juso'][30]
url = 'https://dapi.kakao.com/v2/local/search/address.json?&query=' + address
result = requests.get(urlparse(url).geturl(), headers={'Authorization': 'KakaoAK 0XXX6'}).json()
match_first = result['documents'][0]['address']
lat = float(match_first['y'])
lng = float(match_first['x'])
print(address, "//" ,lat,lng)  # 위도(lat) 경도(long)

In [ ]:
# 5-05_데이터베이스 리스트 파일 만들기

apt_list = []
end = len(apt_juso)-1
end

In [ ]:
# 05-07_add_juso 좌표 딕셔너리 만들기(카카오맵 이용한 지오코딩)

import requests
from urllib.parse import urlparse

for x in range(0, end) :
    try:
        address = apt_juso['juso'][x]
        url = 'https://dapi.kakao.com/v2/local/search/address.json?&query=' + address
        result = requests.get(urlparse(url).geturl(), headers={'Authorization': 'KakaoAK 0XXX6'}).json()
        match_first = result['documents'][0]['address']
        lat = float(match_first['y'])
        lng = float(match_first['x'])   
        txt = (address,lng,lat)
        apt_list.append(txt) 
        print(x, '|',address,'|',lat,'|',lng, end="\r")  # 위도(lat=Y) 경도(long=X)  
    except:  
        print("에러발생")

In [ ]:
# 05-08_리스트를 데이터프레임으로 변환하기

add_juso = pd.DataFrame(apt_list)
add_juso.columns = ['juso', 'y', 'x'] 
add_juso
add_juso.shape

In [ ]:
# 05-11_인덱스 리셋 하기

add_juso = add_juso.reset_index()
add_juso = add_juso.drop('index', axis=1)
add_juso

In [ ]:
# 05-13_플로팅 하기

import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots()
ax = sns.scatterplot(x= 'y', y= 'x', data=add_juso) 

In [ ]:
apt_juso

In [ ]:
add_juso

In [ ]:
# 05-17_주소 coordinate 좌표값 join 하기 

items = pd.merge(items, add_juso, how='left', left_on=['juso'], right_on=['juso'])
items.head()
items.shape

In [ ]:
# 05-18_apt_db 저장하기

items.to_pickle('05_price_DB.pkl')
items.to_csv("05_price_DB.csv", encoding="euc-kr") 
items.head(4)

In [ ]:
# 05-19_메모리 클리어 

%reset